<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import warnings
import requests
from bs4 import BeautifulSoup

In [2]:
#turn off UserWarnings
warnings.filterwarnings('ignore')

In [3]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [4]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [5]:
# текст запроса
query_3_1 = f'''
SELECT COUNT(id) FROM VACANCIES
'''

In [6]:
vacancies = pd.read_sql_query(query_3_1, connection).iloc[0][0]
print("Число вакансий в базе: ", vacancies)

Число вакансий в базе:  49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [7]:
# текст запроса
query_3_2 = f'''
SELECT COUNT(id) FROM EMPLOYERS
'''

In [8]:
employers = pd.read_sql_query(query_3_2, connection).iloc[0][0]
print("Число работодателей в базе: ", employers)

Число работодателей в базе:  23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [9]:
query_3_3 = f'''
SELECT COUNT(id) FROM AREAS
'''

In [10]:
areas = pd.read_sql_query(query_3_3, connection).iloc[0][0]
print("Число регионов: ", areas)

Число регионов:  1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [11]:
query_3_4 = f'''
SELECT COUNT(id) FROM INDUSTRIES
'''

In [12]:
industries = pd.read_sql_query(query_3_4, connection).iloc[0][0]
print("Число сфер деятельности компаний: ", industries)

Число сфер деятельности компаний:  294


***

В базе 23 тысячи работодателей. Из них у 15 тысяч есть открытые вакансии. Всего вакансий в базе - 49 тысяч. В среднем чуть больше 3 вакансий на работодателя.

Число городов/регионов 1362. Число городов в России меньше  - 1118 на октябрь 2021 года. По всей видимости в списке регионов представлены большинство или все города РФ. А также ряд городов из других стран, в основном ближайших, поскольку они с большей вероятностью будут искать работников на платформе из-за географический и языковых причин.

Число сфер деятельности - 294. Представлен широкий спектр вакансий, не сконцентрированный вокруг узкого направления в дятельности. IT-специалисты находят применение себе в самых разных сферах.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [13]:
query_4_1 = f'''
SELECT 
    ar.name AS area,
    COUNT(vac.id) AS cnt 
FROM VACANCIES vac
INNER JOIN areas ar ON vac.area_id = ar.id
GROUP BY ar.name
ORDER BY cnt DESC
'''

In [14]:
vac_areas = pd.read_sql_query(query_4_1, connection)

#top 5 for task 4.1
display(vac_areas.iloc[:5])

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [15]:
query_4_2 = f'''
SELECT 
    COUNT(id) 
FROM VACANCIES
WHERE salary_from IS NOT NULL
    OR salary_to IS NOT NULL
'''

In [16]:
vac_salary = pd.read_sql_query(query_4_2, connection).iloc[0][0]

print("Число вакансий с данными о зарплате: ", vac_salary)

Число вакансий с данными о зарплате:  24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [17]:
query_4_3 = f'''
SELECT 
    ROUND(AVG(salary_from)) as AvgMinSalary,
    ROUND(AVG(salary_to)) as AvgMaxSalary
FROM VACANCIES
'''

In [18]:
avg_salary = pd.read_sql_query(query_4_3, connection)

print("Средняя нижняя граница заработной платы: ", int(avg_salary.iloc[0][0]))
print("Средняя верхняя граница заработной платы: ", int(avg_salary.iloc[0][1]))

Средняя нижняя граница заработной платы:  71065
Средняя верхняя граница заработной платы:  110537


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [19]:
query_4_4 = f'''
SELECT 
    schedule,
    employment,
    COUNT(id) as Count
FROM VACANCIES
GROUP BY schedule, employment
ORDER BY Count DESC
'''

In [20]:
shedule_empl = pd.read_sql_query(query_4_4, connection)

display(shedule_empl)

#Answer for task 4.4
print("После наиболее популярного сочетания \"Полный день - Полная занятость\" следует вариант:\n")
display(shedule_empl.iloc[[1]])

,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


После наиболее популярного сочетания "Полный день - Полная занятость" следует вариант:



,schedule,employment,count
1,Удаленная работа,Полная занятость,7802


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [21]:
query_4_5 = f'''
SELECT 
    experience,
    COUNT(id) as Count
FROM VACANCIES
GROUP BY experience
ORDER BY Count ASC
'''

In [22]:
vac_experience = pd.read_sql_query(query_4_5, connection)

display(vac_experience)

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

Ранее было определено, что 49 тысяч вакансий распределены в 1362 регионах. Если посмотреть детальнее, то заметно, что треть всех вакансий находятся в пяти крупнейших городах. В большинстве же регионов не наберётся и десятка вакансий. Медианное значение - 4 вакансии на регион.

Половина всех вакансий без указания заработной платы. Это даёт свободу работодателям в предложении по результатам собеседования,а также позволяет получить больший выбор в соискателях, которых не отпугнёт заявленная оплататруда. Соискатели же вынуждены тратить время на вакансии, которые им не подойдут по итогу.

Наиболее частым характером работы является классический "полная занятость - полный день" в офисе. Однако довольно значительное число вакансий включают возможность удалённой работы, хотя их и меньше в 4.5 раза.

Работодатели в первую очередь заинтересованы в специалистах, уже знакомых с выполняемыми обязанностями (больше года опыта). В половину меньшее число вакансий ожидают от соискателя уровня уверенных профессионалов. Специалисты же без опыта мало кого интересуют. Начинающему специалисту тяжело начать, но послеполучения первого рабочего опыта по специальности для него(неё) открываются широкие возможности для трудоустройства.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [55]:
query_5_1 = f'''
SELECT 
    emp.name,
    COUNT(vac.id) as VacCount
FROM vacancies vac
INNER JOIN employers emp ON vac.employer_id = emp.id
GROUP BY emp.name
ORDER BY VacCount DESC
LIMIT 10
'''

In [56]:
top_employers = pd.read_sql_query(query_5_1, connection)


print("Первый по числу вакансий работодатель", top_employers.iloc[0][0], "с числом вакансий", top_employers.iloc[0][1])
print("Пятый по числу вакансий работодатель", top_employers.iloc[4][0], "с числом вакансий", top_employers.iloc[4][1])

Первый по числу вакансий работодатель Яндекс с числом вакансий 1933
Пятый по числу вакансий работодатель Газпром нефть с числом вакансий 331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [53]:
query_5_2 = f'''
SELECT
    ar.name,
    COUNT(emp.id) AS EmpCount,
    COUNT(vac.id) AS VacCount 
FROM areas ar
LEFT JOIN vacancies vac ON ar.id = vac.area_id
LEFT JOIN employers emp ON emp.area = ar.id
GROUP BY ar.name
HAVING COUNT(vac.id) = 0
ORDER BY EmpCount DESC
LIMIT 1
'''

In [26]:
top_area_emp = pd.read_sql_query(query_5_2, connection).iloc[0][0]

print("Регион с наибольшим числом работодателей, но без вакансий:", top_area_emp)

Регион с наибольшим числом работодателей, но без вакансий: Россия


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [27]:
query_5_3 = f'''
SELECT 
    emp.name AS EmpName,
    COUNT(DISTINCT(vac.area_id)) AS AreaCount
FROM vacancies vac
INNER JOIN employers emp ON vac.employer_id = emp.id
GROUP BY emp.name
ORDER BY AreaCount DESC
'''

In [28]:
emp_everywhere = pd.read_sql_query(query_5_3, connection)

print("Работодатели, имеющие вакансии в наибольшем числе регионов")

display(emp_everywhere.iloc[:10])

top_emp_everywhere = emp_everywhere.iloc[0]

print(f"Наиболее представленный в регионах работодатель - {top_emp_everywhere.iloc[0]} с числом регионов в {top_emp_everywhere.iloc[1]}")

Работодатели, имеющие вакансии в наибольшем числе регионов


,empname,areacount
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
5,Совкомбанк,63
6,МТС,55
7,"ЭФКО, Управляющая компания",49
8,Почта России,48
9,КРОН,48


Наиболее представленный в регионах работодатель - Яндекс с числом регионов в 181


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [29]:
query_5_4 = f'''
SELECT 
    COUNT(DISTINCT(emp.id))
FROM EMPLOYERS emp
LEFT JOIN EMPLOYERS_INDUSTRIES emi ON emp.id = emi.employer_id
WHERE industry_id IS NULL
'''

In [30]:
emp_no_industry = pd.read_sql_query(query_5_4, connection)

print(f"Число работодателей без указанной сферы деятельности: {emp_no_industry.iloc[0][0]}")

Число работодателей без указанной сферы деятельности: 8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [31]:
query_5_5 = f'''
SELECT 
    emp.name,
    COUNT(industry_id) as IndustryCount
FROM EMPLOYERS emp
LEFT JOIN employers_industries emi ON emp.id = emi.employer_id
GROUP BY emp.name
HAVING  COUNT(industry_id) = 4
ORDER BY name ASC
'''

In [32]:
four_industries = pd.read_sql_query(query_5_5, connection)

print(f"Третье место в алфавитном списке работодателей с четырьмя сферами деятельности занимает \"{four_industries.iloc[2][0]}\"")

Третье место в алфавитном списке работодателей с четырьмя сферами деятельности занимает "2ГИС"


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [33]:
query_5_6 = f'''
SELECT 
    COUNT(emp.id)
FROM employers emp
LEFT JOIN EMPLOYERS_INDUSTRIES emi ON emp.id = emi.employer_id
LEFT JOIN INDUSTRIES ind ON emi.industry_id = ind.id
WHERE ind.name = 'Разработка программного обеспечения'
'''

In [34]:
soft_developers = pd.read_sql_query(query_5_6, connection)

print(f"Число работодателей в сфере разработки ПО: {soft_developers.iloc[0][0]}")

Число работодателей в сфере разработки ПО: 3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [35]:
url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России' # Определяем адрес страницы
response = requests.get(url)

page = BeautifulSoup(response.text, 'html.parser')

millionaires = []

city_column = page.find('table', class_="standard sortable").findAll('td')
for raw in city_column:
    if raw.find('a') is not None:
        millionaires.append(raw.find('a').text)
        
millionaires = str(millionaires)[1:-1]

In [36]:
query_5_7 = f'''
SELECT 
    ar.name,
    COUNT(vac.id)
FROM vacancies vac
LEFT JOIN employers emp ON vac.employer_id = emp.id
LEFT JOIN areas ar ON vac.area_id = ar.id
WHERE ar.name IN ({millionaires})
AND emp.name = 'Яндекс'
GROUP BY ar.name,emp.name

UNION ALL

SELECT 
    'Total',
    COUNT(vac.id)
FROM vacancies vac
LEFT JOIN employers emp ON vac.employer_id = emp.id
LEFT JOIN areas ar ON vac.area_id = ar.id
WHERE ar.name IN ({millionaires})
AND emp.name = 'Яндекс'
'''

In [37]:
yandex_areas = pd.read_sql_query(query_5_7, connection).sort_values('count', ignore_index=True)
display(yandex_areas)

print(f"Число строк в выгрузке: {yandex_areas.shape[0]}")
print(f"Общее число вакансий для городов-миллионников: {yandex_areas.iloc[-1][1]}")

,name,count
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Казань,25
5,Ростов-на-Дону,25
6,Пермь,25
7,Самара,26
8,Уфа,26
9,Краснодар,30


Число строк в выгрузке: 17
Общее число вакансий для городов-миллионников: 485


***

В базе 23.5 тысячи работодателей, из которых 3500 действуют в сфере разработки ПО. Из всех предстваленных работодателей у 15 тысяч есть открытые вакансии. Всего вакансий в базе - 49 тысяч. В среднем чуть больше 3 вакансий на работодателя. Но больше половины работодателей имеют одну вакансию, а 80% компаний имеют до трёх открытых вакансий. Крупных компаний с большим числом вакансий относительно немного. Наиболее заметный рабтодатель среди них - Яндекс. Компания набирает сотрудников в 180 регионах. Также из 12.5 тысяч вакансий компаний в сфере разработки ПО почти две тысячи приходится на долю Яндекса, который предлагает вакансий больше, чем пять других крупнейших работодателей.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [38]:
query_6_1 = f'''
SELECT
    COUNT(vac.id)
FROM vacancies vac
WHERE LOWER(name) LIKE '%data%'
OR LOWER(name) LIKE '%данн%'
'''

In [39]:
data_vacancies = pd.read_sql_query(query_6_1, connection)

print(f"Число вакансий в сфере данных: {data_vacancies.iloc[0][0]}")

Число вакансий в сфере данных: 1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [40]:
query_6_2 = f'''
SELECT
    COUNT(vac.id)
FROM vacancies vac
WHERE 
(
    LOWER(name) LIKE '%data scientist%'
    OR LOWER(name) LIKE '%data science%'
    OR LOWER(name) LIKE '%исследователь данных%'
    OR LOWER(name) LIKE '%machine learning%'
    OR LOWER(name) LIKE '%машинн%обучен%%'
    OR (name LIKE '%ML%' AND name NOT LIKE '%HTML%') 
)
AND
(
    LOWER(name) LIKE '%junior%'
    OR experience = 'Нет опыта'   
    OR employment = 'Стажировка'
)
'''

In [41]:
jun_ds = pd.read_sql_query(query_6_2, connection)

print(f"Число вакансий для начинающих дата-сайентистов: {jun_ds.iloc[0][0]}")

Число вакансий для начинающих дата-сайентистов: 51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

In [42]:
query_6_3 = f'''
SELECT
    COUNT(vac.id)
FROM vacancies vac
WHERE (LOWER(name) LIKE '%data scientist%'
    OR LOWER(name) LIKE '%data science%'
    OR LOWER(name) LIKE '%исследователь данных%'
    OR LOWER(name) LIKE '%machine learning%'
    OR LOWER(name) LIKE '%машинн%обучен%%'
    OR (name LIKE '%ML%' AND name NOT LIKE '%HTML%' 
        --AND name NOT LIKE '%QML%' AND name NOT LIKE '%AML%'
        ) 
)
AND
(LOWER(key_skills) LIKE '%sql%'
    OR LOWER(key_skills) LIKE '%postgres%'
)
'''

In [43]:
sql_vacancies = pd.read_sql_query(query_6_3, connection)

print("Число вакансий data science с ключевым навыком SQL: ", sql_vacancies.iloc[0][0])

Число вакансий data science с ключевым навыком SQL:  201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.


In [44]:
query_6_4 = f'''
SELECT
    COUNT(vac.id)
FROM vacancies vac
WHERE (LOWER(name) LIKE '%data scientist%'
    OR LOWER(name) LIKE '%data science%'
    OR LOWER(name) LIKE '%исследователь данных%'
    OR LOWER(name) LIKE '%machine learning%'
    OR LOWER(name) LIKE '%машинн%обучен%%'
    OR (name LIKE '%ML%' AND name NOT LIKE '%HTML%'
        ) 
)
AND
(
    LOWER(key_skills) LIKE '%python%'
)
'''

In [45]:
python_vacancies = pd.read_sql_query(query_6_4, connection)

print("Число вакансий data science с ключевым навыком Python: ", python_vacancies.iloc[0][0])

Число вакансий data science с ключевым навыком Python:  351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [46]:
query_6_5 = f'''
SELECT
    ROUND(AVG(LENGTH(key_skills) - LENGTH(replace(key_skills,CHR(9),'')) + 1),2)
FROM vacancies vac
WHERE (LOWER(name) LIKE '%data scientist%'
    OR LOWER(name) LIKE '%data science%'
    OR LOWER(name) LIKE '%исследователь данных%'
    OR LOWER(name) LIKE '%machine learning%'
    OR LOWER(name) LIKE '%машинн%обучен%%'
    OR (name LIKE '%ML%' AND name NOT LIKE '%HTML%'
        )
)
'''

In [47]:
skills_vacancies = pd.read_sql_query(query_6_5, connection)

print("Число ключевых навыков для вакансий data science: ", skills_vacancies.iloc[0][0])

Число ключевых навыков для вакансий data science:  6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [48]:
query_6_6 = f'''
SELECT
    experience,
    ROUND(AVG(0.5*COALESCE(salary_from, salary_to) + 0.5*COALESCE(salary_to, salary_from)),0) as avgsalary
FROM vacancies vac
WHERE (LOWER(name) LIKE '%data scientist%'
    OR LOWER(name) LIKE '%data science%'
    OR LOWER(name) LIKE '%исследователь данных%'
    OR LOWER(name) LIKE '%machine learning%'
    OR LOWER(name) LIKE '%машинн%обучен%%'
    OR (name LIKE '%ML%' AND name NOT LIKE '%HTML%'
        )
)
AND (salary_from IS NOT NULL
    OR salary_to IS NOT NULL
)
GROUP BY experience
'''

In [49]:
exp_salary = pd.read_sql_query(query_6_6, connection)
exp_over_three = int(exp_salary[exp_salary['experience'] == 'От 3 до 6 лет'].iloc[0][1])
print(f"Средняя заработная плата специалиста data science с опытом более трёх лет: {exp_over_three} руб.")

Средняя заработная плата специалиста data science с опытом более трёх лет: 243115 руб.


In [58]:
display(exp_salary)

,experience,avgsalary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

В области анализа данных представлены 1800 вакансий. Из них связанные с Data Science - 480. Но требуются в основном, как обычно и бывает, специалисты с опытом. Без опыта только 50 вакансий.

Важными навыками в DST являются SQL и Python. Без них только 53 вакансии. И то эти навыки не указываются зачастую именно из-за того, что знание SQL и скриптовых языков подразумеваются и работодатель указывает более существенные навыки.
Среднее число требований у DST не отличается от других вакансий (6.41 против 6.37).
Заработная плата специалиста DST в два раза выше, чем в среднем по рынку. Но растёт с увеличением опыта в схожей пропорции: без опыта - N; более года - 2N; более трёх - 3N.

# Общий вывод по проекту

Мяса накидать.

Что вообще есть в таблице. Вакансии, работодатели, сферы деятельности, регионы. Распреденление по регионам. По сферам деятельности. По компаниям. У многих компаний не указана сфера деятельности. Поэтому могут быть не совсем точные данные о распределении вакансий по сферам деятельности. 
ЗП где есть какой размер, режим работы.
Анализ данных, Сколько DST, какая ЗП. Формат работы офис/удалёнка тот же. Сферы те же. 

